In [69]:
# Relevant Packages
import pandas as pd
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import pyart
import os

# suppress warnings
import warnings
warnings.filterwarnings("ignore")

### STEP 1

In [86]:
# Read in the lidar data
# 3 Aug, 6 Aug, 7 Aug 2019 are the days with lidar data!
lidar_path = "../lidar_data/lidar-data-3-Aug.csv" # path to file can be changed as needed
dateNum = 3 # date number, to be changed according to the lidar date

lidar = pd.read_csv(lidar_path)
lidar = lidar.drop("Unnamed: 0", axis=1)

# check the lidar data imported
lidar

,Midtime,Latitude,Longitude,Fire ID,Pass #,MLH,Plume Height,MLH Estimate
0,78247.007,47.972,-116.836,10.0,1.0,2143.3048,-9999.0,2850.8139
1,78257.007,47.986,-116.855,10.0,1.0,2712.9095,-9999.0,2850.8139
2,78267.503,48.001,-116.876,10.0,1.0,2802.8473,-9999.0,2850.8139
3,78277.504,48.015,-116.895,10.0,1.0,3042.6808,-9999.0,2850.8139
4,78287.504,48.028,-116.915,10.0,1.0,3102.6393,-9999.0,2850.8139
...,...,...,...,...,...,...,...,...
1631,95231.503,48.461,-117.760,10.0,2.0,3012.7015,-9999.0,3069.1330
1632,95241.503,48.474,-117.761,10.0,2.0,2982.7224,-9999.0,3069.1330
1633,95252.007,48.487,-117.761,10.0,2.0,3012.7015,-9999.0,3069.1330
1634,95262.007,48.499,-117.761,10.0,2.0,3072.6600,-9999.0,3069.1330


In [87]:
# filter out the heights outside of the grid box
# change these values as needed for pre-determined grid
# 47.85 <= latitude <= 48.05
# -118.70 <= longitude <= -118.30

gridbox = lidar[lidar['Latitude'] >= 47.85]
gridbox = gridbox[gridbox['Latitude'] <= 48.05]
gridbox = gridbox[gridbox['Longitude'] >= -118.70]
gridbox = gridbox[gridbox['Longitude'] <= -118.30]

In [88]:
gridbox

,Midtime,Latitude,Longitude,Fire ID,Pass #,MLH,Plume Height,MLH Estimate
66,78921.504,48.041,-118.304,10.0,1.0,-9999.0,2772.8680,2358.4497
67,78931.503,48.034,-118.326,10.0,1.0,-9999.0,2832.8264,2358.4497
68,78942.007,48.027,-118.350,10.0,1.0,-9999.0,3042.6808,2358.4497
69,78952.007,48.020,-118.373,10.0,1.0,-9999.0,3012.7015,2358.4497
70,78962.007,48.012,-118.395,10.0,1.0,-9999.0,2862.8055,2358.4497
...,...,...,...,...,...,...,...,...
1109,89894.007,47.902,-118.356,10.0,2.0,-9999.0,3102.6393,2818.8361
1110,89904.007,47.889,-118.356,10.0,2.0,-9999.0,3102.6393,2818.8361
1111,89914.503,47.876,-118.355,10.0,2.0,-9999.0,3102.6393,2818.8361
1112,89924.504,47.863,-118.355,10.0,2.0,-9999.0,3012.7015,2818.8361


In [89]:
# Convert lidar times
# to datetime format

# Lidar times
ltimes = np.array(gridbox["Midtime"])

# change the unixtime accordingly
unixtime = 1564790400 # 3 August 2019 UTC
# unixtime = 1565049600 # 6 August 2019 UTC
# unixtime = 1565136000 # 7 August 2019 UTC 

# Convert the times to datetime format
ldelta = np.full(len(ltimes), unixtime)
lidartimes = ltimes + ldelta
lidartimes = np.array(lidartimes, dtype="datetime64[s]")

# check the lidar times to make sure they are correct
lidartimes

array(['2019-08-03T21:55:21', '2019-08-03T21:55:31',
       '2019-08-03T21:55:42', '2019-08-03T21:55:52',
       '2019-08-03T21:56:02', '2019-08-03T21:56:12',
       '2019-08-03T21:56:22', '2019-08-03T21:56:32',
       '2019-08-03T21:56:42', '2019-08-03T21:56:52',
       '2019-08-03T21:57:03', '2019-08-03T21:57:13',
       '2019-08-03T21:57:23', '2019-08-03T21:57:33',
       '2019-08-03T21:57:43', '2019-08-03T21:57:53',
       '2019-08-03T21:58:04', '2019-08-03T21:58:14',
       '2019-08-03T22:06:29', '2019-08-03T22:06:40',
       '2019-08-03T22:06:50', '2019-08-03T22:07:00',
       '2019-08-03T22:07:10', '2019-08-03T22:07:20',
       '2019-08-03T22:07:30', '2019-08-03T22:07:41',
       '2019-08-03T22:07:51', '2019-08-03T22:08:01',
       '2019-08-03T22:08:11', '2019-08-03T22:08:21',
       '2019-08-03T22:08:31', '2019-08-03T22:21:39',
       '2019-08-03T22:21:49', '2019-08-03T22:22:00',
       '2019-08-03T22:22:10', '2019-08-03T22:22:20',
       '2019-08-03T22:22:30', '2019-08-03T22:2

In [90]:
# Now check if any lidar times are out of 
# range for the given day

# i.e., if it is 7 Aug 2019 PST - we need to check whether
# it is 8 Aug 2019 UTC. 
N = 0
for i, t in enumerate(lidartimes):
    if t > np.datetime64('2019-08-0'+str(dateNum)+'T23:59:59'):
        N = i
        print("N = ", i, t)
        break

N =  88 2019-08-04T00:19:49


In [91]:
"""
We will change the next_day boolean variable accordingly.
If we are inspecting the heights for 7 Aug UTC, next_day == False
If the heights are for 8 Aug UTC next_day == True
"""
# Comment out accordingly
# next_day = True
next_day = False

if N != 0:
    if next_day == False:
        ltimes = ltimes[:N]
    elif next_day == True:
        ltimes = ltimes[N:]

In [92]:
ltimes # check

array([78921.504, 78931.503, 78942.007, 78952.007, 78962.007, 78972.007,
       78982.503, 78992.503, 79002.504, 79012.504, 79023.007, 79033.007,
       79043.007, 79053.504, 79063.503, 79073.503, 79084.007, 79094.007,
       79589.504, 79600.008, 79610.008, 79620.007, 79630.503, 79640.504,
       79650.503, 79661.007, 79671.007, 79681.008, 79691.504, 79701.504,
       79711.504, 80499.503, 80509.503, 80520.007, 80530.007, 80540.007,
       80550.504, 80560.504, 80570.504, 80581.007, 80591.007, 80601.007,
       80611.503, 80621.504, 80631.503, 80642.007, 80652.007, 80922.007,
       80932.504, 80942.504, 80952.503, 80963.007, 80973.007, 80983.007,
       80993.007, 81003.504, 81013.504, 81023.504, 81034.007, 81044.007,
       81054.007, 81064.504, 81074.504, 81084.504, 81423.504, 81433.504,
       81443.504, 81454.007, 81464.007, 81474.007, 81484.504, 81494.504,
       81504.504, 81515.007, 81525.007, 81535.007, 81545.504, 81555.503,
       81565.503, 81576.008, 81872.007, 81882.504, 

In [93]:
# if you set next_day == True, 
# you need to increase the dateNum by 1
if next_day == True:
    dateNum += 1

In [94]:
# Reading in the Radar data - injection heights within grid box
# change the path accordingly
path = "../Injection-Heights/new-injection-heights_3-Aug-2019.mat"
radar = sio.loadmat(path)

# checking radar data
radar

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Dec  2 17:29:16 2021',
 '__version__': '1.0',
 '__globals__': [],
 'smoke_injection_height': array([[[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],
 
        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],
 
        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ...,
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
        

In [95]:
# reading in the radar times
rtimes = np.array(radar["time"][0])
rtimes # check

array([4.7000e+01, 3.0700e+02, 5.6900e+02, 8.3100e+02, 1.0920e+03,
       1.3540e+03, 1.6160e+03, 1.8760e+03, 2.1370e+03, 2.3980e+03,
       2.6690e+03, 2.9390e+03, 3.2040e+03, 3.4730e+03, 3.7440e+03,
       4.0150e+03, 4.2810e+03, 4.5480e+03, 4.8140e+03, 5.0790e+03,
       5.3450e+03, 5.6120e+03, 5.8840e+03, 6.1490e+03, 6.4160e+03,
       6.6810e+03, 6.9460e+03, 7.2120e+03, 7.4770e+03, 7.7470e+03,
       8.0130e+03, 8.2790e+03, 8.5460e+03, 8.8120e+03, 9.0770e+03,
       9.3420e+03, 9.6030e+03, 9.8630e+03, 1.0124e+04, 1.0386e+04,
       1.0646e+04, 1.0907e+04, 1.1168e+04, 1.1429e+04, 1.1689e+04,
       1.1964e+04, 1.2239e+04, 1.2514e+04, 1.2789e+04, 1.3066e+04,
       1.3336e+04, 1.3607e+04, 1.3877e+04, 1.4153e+04, 1.4428e+04,
       1.4711e+04, 1.4993e+04, 1.5274e+04, 1.5555e+04, 1.5837e+04,
       1.6119e+04, 1.6402e+04, 1.6685e+04, 1.6966e+04, 1.7247e+04,
       1.7528e+04, 1.7810e+04, 1.8091e+04, 1.8373e+04, 1.8654e+04,
       1.8935e+04, 1.9217e+04, 1.9498e+04, 1.9781e+04, 2.0062e

In [96]:
# if we want the next day times, add 24 hours to the times of the following day
if next_day == True:
    rtimes += (24*60*60)
    
rtimes # check

array([4.7000e+01, 3.0700e+02, 5.6900e+02, 8.3100e+02, 1.0920e+03,
       1.3540e+03, 1.6160e+03, 1.8760e+03, 2.1370e+03, 2.3980e+03,
       2.6690e+03, 2.9390e+03, 3.2040e+03, 3.4730e+03, 3.7440e+03,
       4.0150e+03, 4.2810e+03, 4.5480e+03, 4.8140e+03, 5.0790e+03,
       5.3450e+03, 5.6120e+03, 5.8840e+03, 6.1490e+03, 6.4160e+03,
       6.6810e+03, 6.9460e+03, 7.2120e+03, 7.4770e+03, 7.7470e+03,
       8.0130e+03, 8.2790e+03, 8.5460e+03, 8.8120e+03, 9.0770e+03,
       9.3420e+03, 9.6030e+03, 9.8630e+03, 1.0124e+04, 1.0386e+04,
       1.0646e+04, 1.0907e+04, 1.1168e+04, 1.1429e+04, 1.1689e+04,
       1.1964e+04, 1.2239e+04, 1.2514e+04, 1.2789e+04, 1.3066e+04,
       1.3336e+04, 1.3607e+04, 1.3877e+04, 1.4153e+04, 1.4428e+04,
       1.4711e+04, 1.4993e+04, 1.5274e+04, 1.5555e+04, 1.5837e+04,
       1.6119e+04, 1.6402e+04, 1.6685e+04, 1.6966e+04, 1.7247e+04,
       1.7528e+04, 1.7810e+04, 1.8091e+04, 1.8373e+04, 1.8654e+04,
       1.8935e+04, 1.9217e+04, 1.9498e+04, 1.9781e+04, 2.0062e

In [97]:
# Helper function to find nearest time
def nearest(times, tvalue):
    tarr = np.asarray(times)
    idx = (np.abs(tarr - tvalue)).argmin()
    return times[idx], idx

In [98]:
radartimes = np.array([])
rindexes = np.array([])
for t in np.asarray(ltimes):
    r, i = nearest(rtimes, t)
    print(r, i) # check
    radartimes = np.append(radartimes, r)
    rindexes = np.append(rindexes, i)

78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
78985.0 238
79413.0 239
79413.0 239
79413.0 239
79413.0 239
79842.0 240
79842.0 240
79842.0 240
79842.0 240
79842.0 240
79842.0 240
79842.0 240
79842.0 240
79842.0 240
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
80698.0 242
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81121.0 243
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
81550.0 244
82092.0 245
82092.0 245
82092.0 245
8209

In [99]:
#reorganized radar and lidar data
lidar = gridbox
rdata = pd.DataFrame() # empty data frame

if N!= 0:
    if next_day == False:  
        rdata['lidar_times'] = np.asarray(ltimes)
        rdata['radar_times'] = radartimes
        rdata['time_index'] = rindexes
        rdata['lidar_heights'] = np.asarray(lidar["Plume Height"][:N])
        rdata['lat'] = np.asarray(lidar["Latitude"][:N])
        rdata['long'] = np.asarray(lidar['Longitude'][:N])
    else:
        rdata['lidar_times'] = np.asarray(ltimes)
        rdata['radar_times'] = radartimes
        rdata['time_index'] = rindexes
        rdata['lidar_heights'] = np.asarray(lidar["Plume Height"][N:])
        rdata['lat'] = np.asarray(lidar["Latitude"][N:])
        rdata['long'] = np.asarray(lidar['Longitude'][N:])
else:
    rdata['lidar_times'] = np.asarray(ltimes)
    rdata['radar_times'] = radartimes
    rdata['time_index'] = rindexes
    rdata['lidar_heights'] = np.asarray(lidar["Plume Height"])
    rdata['lat'] = np.asarray(lidar["Latitude"])
    rdata['long'] = np.asarray(lidar['Longitude'])
        
rdata # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long
0,78921.504,78985.0,238.0,2772.8680,48.041,-118.304
1,78931.503,78985.0,238.0,2832.8264,48.034,-118.326
2,78942.007,78985.0,238.0,3042.6808,48.027,-118.350
3,78952.007,78985.0,238.0,3012.7015,48.020,-118.373
4,78962.007,78985.0,238.0,2862.8055,48.012,-118.395
...,...,...,...,...,...,...
83,81902.503,82092.0,245.0,3402.4313,48.002,-118.325
84,81913.007,82092.0,245.0,3402.4313,48.014,-118.334
85,81923.007,82092.0,245.0,3492.3688,48.025,-118.342
86,81933.007,82092.0,245.0,3282.5144,48.036,-118.350


In [19]:
# Importing raw radar data files
# change this as needed:
path = "../../../data/3aug2019"
filenames = []
for root, dirs, files in os.walk(path):
    for f in sorted(files):
        for i in range(10):
            if f.startswith('KOTX2019080'+str(dateNum)+'_0'+str(i)):
                filenames.append(f)
        for j in range(10, 24):
            if f.startswith('KOTX2019080'+str(dateNum)+'_'+str(j)):
                filenames.append(f)

In [20]:
len(filenames)

256

In [21]:
# Helper function to find nearest latitude longitude position
def nearest_latlong(latitudes, longitudes, lidarlat, lidarlong):
    xval = 0
    yval = 0
    totalmin = 1000

    # actual distance 
    difflat2 = np.power(np.abs(latitudes - lidarlat), 2)
    difflong2 = np.power(np.abs(longitudes - lidarlong), 2)

    diff = np.sqrt(difflat2+difflong2)

    [xval, yval] = np.where(np.min(diff)==diff)
                
#     print(xval[0], yval[0])
#     print(latitudes.shape)
    # returning the indexes of the latitude
    return latitudes[xval, yval][0], longitudes[xval, yval][0], xval[0], yval[0]

In [22]:
# Initializing arrays
radar_latitudes = np.zeros(len(rdata))
radar_longitudes = np.zeros(len(rdata))
reflectivities = np.zeros(100*len(rdata)).reshape(len(rdata), 100)
correlations = np.zeros(100*len(rdata)).reshape(len(rdata), 100)
heights = np.zeros(100*len(rdata)).reshape(len(rdata), 100)
injections = np.zeros(len(rdata))

# Remember to comment the following out if you need to reload the saved data:
# radar_latitudes = savedfile['latitudes'][0]
# radar_longitudes = savedfile['longitudes'][0]
# reflectivities = savedfile['reflectivity']
# correlations = savedfile['correlation']
# heights = savedfile['altitudes']
# injections = savedfile['injection_heights'][0]

In [23]:
# last file saved
saved = 0

# Lidar Location Mapping

for row in range(saved, len(rdata)):
    fnum = int(rdata["time_index"][row])
    file = filenames[fnum]
    print('file-num', fnum, 'rdata-', row)

    if row != 0:
        if int(rdata["time_index"][row-1]) != fnum:
            # Radar object and regridding
            radar = pyart.io.read(path+file) # removed scans
            grid = pyart.map.grid_from_radars((radar,), grid_shape=(100,200,200),
                                             grid_limits=((0,10000), (-125000.0, 125000.0), (-125000.0, 125000.0)))
    else:
        # Radar object and regridding
        radar = pyart.io.read(path+file) # removed scans
        grid = pyart.map.grid_from_radars((radar,), grid_shape=(100,200,200),
                                         grid_limits=((0,10000), (-125000.0, 125000.0), (-125000.0, 125000.0)))

    print("passed") # checkpoint
    
    lidarlat = rdata['lat'][row]
    lidarlong = rdata['long'][row]

    # Latitude, Longitude, Altitude
    latitudes = grid.point_latitude['data'][0, :, :]
    longitudes = grid.point_longitude['data'][0, :, :]
    altitudes = grid.point_altitude['data'][:, :, :]
    
    # Reflectivity
    refs = grid.fields['reflectivity']['data'][:, :, :].data
    # Correlation Coefficient
    coeffs = grid.fields['cross_correlation_ratio']['data'][:, :, :].data
    
    # Nearest Lat and Long
    radarlat, radarlong, rx, ry = nearest_latlong(latitudes, longitudes, lidarlat, lidarlong)
    radar_latitudes[row] = radarlat
    radar_longitudes[row] = radarlong
    
    # Associated refs, coeffs, and heights
    reflectivities[row] = refs[:, rx, ry]
    correlations[row] = coeffs[:, rx, ry]
    heights[row] = altitudes[:, rx, ry]
    
    # Injection Height Extraction
    bad_count = 0
    current_height = -1
    for i in range(len(reflectivities[row])):
        
        ref = reflectivities[row][i]
        coef = correlations[row][i]
        
        if ref >= 10.000 and 0.2 < coef and coef < 0.9:
            current_height = heights[row][i]
        else:
            bad_count += 1
            
        if bad_count > 2:
            injections[row] = current_height
            break
    
    savedict = {
        "reflectivity" : reflectivities,
        "correlation" : correlations,
        "altitudes" : heights,
        "latitudes" : radar_latitudes,
        "longitudes" : radar_longitudes,
        "injection_heights" : injections
    }
    sio.savemat('gridbox_'+str(dateNum)+'Aug2019.mat', savedict)
    
    print('done')

file-num 14 rdata- 0
passed
done
file-num 14 rdata- 1
passed
done
file-num 14 rdata- 2
passed
done
file-num 14 rdata- 3
passed
done
file-num 14 rdata- 4
passed
done
file-num 14 rdata- 5
passed
done
file-num 14 rdata- 6
passed
done
file-num 14 rdata- 7
passed
done
file-num 14 rdata- 8
passed
done
file-num 14 rdata- 9
passed
done
file-num 14 rdata- 10
passed
done
file-num 14 rdata- 11
passed
done
file-num 14 rdata- 12
passed
done
file-num 14 rdata- 13
passed
done
file-num 14 rdata- 14
passed
done
file-num 14 rdata- 15
passed
done
file-num 14 rdata- 16
passed
done
file-num 16 rdata- 17
passed
done
file-num 16 rdata- 18
passed
done
file-num 16 rdata- 19
passed
done
file-num 16 rdata- 20
passed
done
file-num 16 rdata- 21
passed
done
file-num 16 rdata- 22
passed
done
file-num 16 rdata- 23
passed
done
file-num 16 rdata- 24
passed
done
file-num 16 rdata- 25
passed
done
file-num 16 rdata- 26
passed
done
file-num 16 rdata- 27
passed
done
file-num 16 rdata- 28
passed
done


Before proceeding with Step 2, go back and re-do step 1 for the other days you would need to compare. You will need to create and save seperate .mat files of the data for a given day.

### STEP 2

In [122]:
aug3 = sio.loadmat('gridbox_3Aug2019.mat')
aug4 = sio.loadmat('gridbox_4Aug2019.mat')
aug6 = sio.loadmat('gridbox_6Aug2019.mat')
aug7 = sio.loadmat('gridbox_7Aug2019.mat')
aug8 = sio.loadmat('gridbox_8Aug2019.mat')

In [191]:
# lidar data for August 3, 2019
# Read in the lidar data
# 3 Aug, 6 Aug, 7 Aug 2019 are the days with lidar data!
lidar_path = "../lidar_data/lidar-data-7-Aug.csv" # path to file can be changed as needed
dateNum = 7 # date number, to be changed according to the lidar date

lidar = pd.read_csv(lidar_path)
lidar = lidar.drop("Unnamed: 0", axis=1)

# filter out the heights outside of the grid box
# change these values as needed for pre-determined grid
# 47.85 <= latitude <= 48.05
# -118.70 <= longitude <= -118.30
gridbox = lidar[lidar['Latitude'] >= 47.85]
gridbox = gridbox[gridbox['Latitude'] <= 48.05]
gridbox = gridbox[gridbox['Longitude'] >= -118.70]
gridbox = gridbox[gridbox['Longitude'] <= -118.30]

# Lidar times
ltimes = np.array(gridbox["Midtime"])

# change the unixtime accordingly
# unixtime = 1564790400 # 3 August 2019 UTC
# unixtime = 1565049600 # 6 August 2019 UTC
unixtime = 1565136000 # 7 August 2019 UTC 

# Convert the times to datetime format
ldelta = np.full(len(ltimes), unixtime)
lidartimes = ltimes + ldelta
lidartimes = np.array(lidartimes, dtype="datetime64[s]")

# Now check if any lidar times are out of 
# range for the given day

# i.e., if it is 7 Aug 2019 PST - we need to check whether
# it is 8 Aug 2019 UTC. 
N = 0
for i, t in enumerate(lidartimes):
    if t > np.datetime64('2019-08-0'+str(dateNum)+'T23:59:59'):
        N = i
        print("N = ", i, t)
        break

N =  36 2019-08-08T01:01:02


In [192]:
"""
We will change the next_day boolean variable accordingly.
If we are inspecting the heights for 7 Aug UTC, next_day == False
If the heights are for 8 Aug UTC next_day == True
"""
# Change bool accordingly
next_day = True

if N != 0:
    if next_day == False:
        ltimes = ltimes[:N]
    elif next_day == True:
        ltimes = ltimes[N:]
        
if next_day == True:
    dateNum += 1

In [194]:
# Reading in the Radar data - injection heights within grid box
# change the path accordingly
path = "../Injection-Heights/new-injection-heights_8-Aug-2019.mat"
radar = sio.loadmat(path)

# reading in the radar times
rtimes = np.array(radar["time"][0])
# if we want the next day times, add 24 hours to the times of the following day
if next_day == True:
    rtimes += (24*60*60)

radartimes = np.array([])
rindexes = np.array([])
for t in np.asarray(ltimes):
    r, i = nearest(rtimes, t)
    radartimes = np.append(radartimes, r)
    rindexes = np.append(rindexes, i)

In [195]:
#reorganized radar and lidar data
lidar = gridbox
rdata = pd.DataFrame() # empty data frame

if N!= 0:
    if next_day == False:  
        rdata['lidar_times'] = np.asarray(ltimes)
        rdata['radar_times'] = radartimes
        rdata['time_index'] = rindexes
        rdata['lidar_heights'] = np.asarray(lidar["Plume Height"][:N])
        rdata['lat'] = np.asarray(lidar["Latitude"][:N])
        rdata['long'] = np.asarray(lidar['Longitude'][:N])
    else:
        rdata['lidar_times'] = np.asarray(ltimes)
        rdata['radar_times'] = radartimes
        rdata['time_index'] = rindexes
        rdata['lidar_heights'] = np.asarray(lidar["Plume Height"][N:])
        rdata['lat'] = np.asarray(lidar["Latitude"][N:])
        rdata['long'] = np.asarray(lidar['Longitude'][N:])
else:
    rdata['lidar_times'] = np.asarray(ltimes)
    rdata['radar_times'] = radartimes
    rdata['time_index'] = rindexes
    rdata['lidar_heights'] = np.asarray(lidar["Plume Height"])
    rdata['lat'] = np.asarray(lidar["Latitude"])
    rdata['long'] = np.asarray(lidar['Longitude'])
        
rdata # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long
0,90062.007,90151.0,14.0,-9999.0000,48.005,-118.301
1,90072.007,90151.0,14.0,-9999.0000,48.005,-118.325
2,90082.503,90151.0,14.0,-9999.0000,48.004,-118.350
3,90092.504,90151.0,14.0,-9999.0000,48.003,-118.373
4,90102.504,90151.0,14.0,-9999.0000,48.002,-118.397
5,90112.504,90151.0,14.0,-9999.0000,48.001,-118.421
6,90122.504,90151.0,14.0,-9999.0000,48.000,-118.444
7,90133.007,90151.0,14.0,-9999.0000,48.000,-118.469
8,90143.007,90151.0,14.0,5500.9753,47.999,-118.492
9,90153.007,90151.0,14.0,5051.2871,47.998,-118.516


In [196]:
# change the injection height day accordingly!
rdata['radar_heights'] = aug8['injection_heights'][0]

rdata.head() # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights
0,90062.007,90151.0,14.0,-9999.0,48.005,-118.301,6080.535354
1,90072.007,90151.0,14.0,-9999.0,48.005,-118.325,6181.545455
2,90082.503,90151.0,14.0,-9999.0,48.004,-118.350,6080.535354
3,90092.504,90151.0,14.0,-9999.0,48.003,-118.373,6989.626263
4,90102.504,90151.0,14.0,-9999.0,48.002,-118.397,6282.555556


In [197]:
# re-writing the data
rdata['lidar_times'] = np.round(rdata['lidar_times']/3600, 2)
rdata['radar_times'] = np.round(rdata['radar_times']/3600, 2)

In [198]:
rdata.head() # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights
0,25.02,25.04,14.0,-9999.0,48.005,-118.301,6080.535354
1,25.02,25.04,14.0,-9999.0,48.005,-118.325,6181.545455
2,25.02,25.04,14.0,-9999.0,48.004,-118.350,6080.535354
3,25.03,25.04,14.0,-9999.0,48.003,-118.373,6989.626263
4,25.03,25.04,14.0,-9999.0,48.002,-118.397,6282.555556


In [199]:
# Now, remove all the negative values
compare = rdata[rdata['lidar_heights'] > 0]
compare = compare[compare['radar_heights'] > 0]

compare # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights


In [200]:
diff = compare['radar_heights'] - compare['lidar_heights']
avg = (compare['radar_heights'] + compare['lidar_heights'])/2

percent_diff = 100*(diff/avg)
percent_diff

Series([], dtype: float64)

### Step 2.5
Create a data frame to store percentage differences per day. Remember to create a different cell each time to append the above results to different data frames.

#### August 4, 2019 UTC

In [161]:
aug4utc = compare
aug4utc['% diff'] = percent_diff
aug4utc.head()

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights,% diff
3,24.34,24.34,2.0,4002.0153,47.991,-118.378,3656.292929,-9.028688
4,24.34,24.34,2.0,4331.7864,47.984,-118.403,3959.323232,-8.984640
6,24.35,24.34,2.0,4002.0153,47.974,-118.452,4060.333333,1.446676
9,24.36,24.34,2.0,3132.6184,47.961,-118.527,3858.313131,20.761031
10,24.36,24.34,2.0,3312.4935,47.957,-118.551,3555.282828,7.070391


#### August 3, 2019 UTC

In [150]:
aug3utc = compare
aug3utc['% diff'] = percent_diff
aug3utc.head()

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights,% diff
9,21.95,21.94,238.0,3192.5768,47.976,-118.508,3656.292929,13.541391
11,21.95,21.94,238.0,3222.5560,47.962,-118.554,4161.343434,25.427958
36,22.38,22.42,242.0,3192.5768,47.982,-118.543,4363.363636,30.989838
37,22.38,22.42,242.0,3132.6184,47.970,-118.534,3858.313131,20.761031
38,22.38,22.42,242.0,3162.5975,47.957,-118.526,3555.282828,11.690751


#### August 7, 2019 UTC

In [190]:
aug7utc = compare
aug7utc['% diff'] = percent_diff
aug7utc.head()

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights,% diff
0,23.25,23.25,298.0,7029.9143,48.023,-118.300,4969.424242,-34.343394
7,23.27,23.25,298.0,5470.9961,48.021,-118.465,4565.383838,-18.046592
8,23.27,23.25,298.0,5261.1416,48.017,-118.488,4262.353535,-20.975242
9,23.27,23.25,298.0,4121.9319,48.013,-118.510,2848.212121,-36.547875
34,23.60,23.58,302.0,4721.5159,48.018,-118.306,5575.484848,16.586751


### Step 3
We will now compute an overall percentage difference in heights.

### Aug 3 2019 PST
Combine both of the above dataframes

In [168]:
# concatenate along x
aug3pst = pd.concat([aug3utc, aug4utc], axis=0, ignore_index=True)

aug3pst.head() # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights,% diff
0,21.95,21.94,238.0,3192.5768,47.976,-118.508,3656.292929,13.541391
1,21.95,21.94,238.0,3222.5560,47.962,-118.554,4161.343434,25.427958
2,22.38,22.42,242.0,3192.5768,47.982,-118.543,4363.363636,30.989838
3,22.38,22.42,242.0,3132.6184,47.970,-118.534,3858.313131,20.761031
4,22.38,22.42,242.0,3162.5975,47.957,-118.526,3555.282828,11.690751


In [170]:
aug3pst['% diff'].mean() # overall percentage difference

-0.03791737703516224

### August 7, 2019 PST

In [204]:
aug7pst = aug7utc.reset_index(drop='True')
aug7pst.head() # check

,lidar_times,radar_times,time_index,lidar_heights,lat,long,radar_heights,% diff
0,23.25,23.25,298.0,7029.9143,48.023,-118.300,4969.424242,-34.343394
1,23.27,23.25,298.0,5470.9961,48.021,-118.465,4565.383838,-18.046592
2,23.27,23.25,298.0,5261.1416,48.017,-118.488,4262.353535,-20.975242
3,23.27,23.25,298.0,4121.9319,48.013,-118.510,2848.212121,-36.547875
4,23.60,23.58,302.0,4721.5159,48.018,-118.306,5575.484848,16.586751


In [205]:
aug7pst['% diff'].mean() # overall percentage difference

-5.402528994329498